In [1]:
!pip install gym
!pip install pygame

     |████████████████████████████████| 21.8 MB 1.5 MB/s 


In [2]:
import gym
import numpy as np
import random
from IPython.display import clear_output

In [3]:
env = gym.make("Taxi-v3")

In [9]:
env.reset()
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [13]:
env.reset()

67

### Actions
    There are 6 discrete deterministic actions:
    - 0: move south
    - 1: move north
    - 2: move east
    - 3: move west
    - 4: pickup passenger
    - 5: drop off passenger

In [53]:
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [72]:
# untuk melakukan aksi, pakai env.action({action nya})
print(env.step(0))
env.render()

(479, -1, False, {'prob': 1.0})
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)


In [52]:
env.reset()

403

In [40]:
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [41]:
env.encode(4,1,1,2)

426

In [43]:
print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

Action Space Discrete(6)
State Space Discrete(500)


In [57]:
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)


In [59]:
env.s # ini state nya

433

In [73]:
env.P[479]

{0: [(1.0, 479, -1, False)],
 1: [(1.0, 379, -1, False)],
 2: [(1.0, 499, -1, False)],
 3: [(1.0, 479, -1, False)],
 4: [(1.0, 479, -10, False)],
 5: [(1.0, 475, 20, True)]}

## Selesaikan secara manual, tanpa Reinforced Learning(Random)

In [137]:
episode = 0
penalties, reward = 0,0

frames = []

done = False
env.reset()
while not done:
  action = env.action_space.sample()
  state, reward, done, info = env.step(action)

  if reward == -10:
    penalties +=1
  
  frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )
  
  episode += 1

print("Timesteps taken: {}".format(episode))
print("Penalties incurred: {}".format(penalties))

Timesteps taken: 200
Penalties incurred: 64


In [93]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)
        
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 200
State: 241
Action: 5
Reward: -10


# Use Q Learning

In [138]:
# Values for Q Table:

action_size = env.action_space.n
print('Action Space: ', action_size)

state_size = env.observation_space.n
print('State Size: ', state_size)

Action Space:  6
State Size:  500


In [141]:
# Bikin array kosong seukuran action x state
q_table = np.zeros((state_size, action_size))
q_table.shape

(500, 6)

In [143]:
# Hyper params :

total_ep = 1500
total_test_ep = 100
max_steps = 100

lr = 0.81
gamma = 0.96

#Exploration params :
epsilon = 0.9
max_epsilon = 1.0
min_epsilon = 0.01
decay_rate = 0.01

In [145]:
#akan mengisi Q Table dari hasil learning
for episode in range(total_ep):

  #Reset
  state = env.reset()
  step = 0
  done = False

  for step in range(max_steps):
    #tentukan apakah dia akan melakukan eksplorasi atau eksploitasi
    exp_tradeoff = random.uniform(0,1)

    if exp_tradeoff > epsilon:
      action = np.argmax(q_table[state,:])
    
    else:
      action = env.action_space.sample()

    new_state, reward, done, info = env.step(action)

    q_table[state, action] = q_table[state, action] + lr * (reward + gamma * np.max(q_table[new_state, :]) - q_table[state, action])

    state = new_state

    if done == True:
      break

  episode += 1

  # Reduce epsilon (because we need less and less exploration):
  epsilon = min_epsilon + (max_epsilon - min_epsilon) *np.exp(-decay_rate*episode)

In [146]:
q_table

array([[  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [ -5.36261141,  -5.81490338,  -5.53610013,   2.96180536,
          6.16402981, -12.91245932],
       [ -4.85028504,  -4.84829046,  -4.30676877,   7.40403674,
         11.69177139, -10.561266  ],
       ...,
       [ -3.11848816,  12.75654671,  -3.3721972 ,  -3.11012886,
        -11.97484416,  -9.639     ],
       [ -5.52888145,  -5.09863594,  -5.52977677,  -3.94182101,
        -11.93219878,  -8.1       ],
       [ -1.593756  ,  -1.86789807,  -1.593756  ,  18.19998784,
        -10.268856  ,   4.01997016]])

# Using the Q Table

In [148]:
env.reset()
rewards = []

for episode in range(total_test_ep):
  state = env.reset()
  step = 0
  done = False
  total_rewards = 0
  print('=========================')
  print('EPISODE: ', episode)

  for step in range(max_steps):

    #env.render()

    # Take the action based on the Q Table:
    action = np.argmax(q_table[state, :])

    new_state, reward, done, info = env.step(action)

    total_rewards += reward

    # If episode finishes:
    if done:
      rewards.append(total_rewards)
      print('Score: ', total_rewards)
      print('State count :', step)
      break

    state = new_state

env.close()
print('Score Over Time: {}'.format(sum(rewards)/total_test_ep))

EPISODE:  0
Score:  6
State count : 14
EPISODE:  1
Score:  9
State count : 11
EPISODE:  2
Score:  3
State count : 17
EPISODE:  3
Score:  13
State count : 7
EPISODE:  4
Score:  8
State count : 12
EPISODE:  5
Score:  3
State count : 17
EPISODE:  6
Score:  10
State count : 10
EPISODE:  7
Score:  8
State count : 12
EPISODE:  8
Score:  13
State count : 7
EPISODE:  9
Score:  7
State count : 13
EPISODE:  10
Score:  10
State count : 10
EPISODE:  11
Score:  11
State count : 9
EPISODE:  12
Score:  8
State count : 12
EPISODE:  13
Score:  11
State count : 9
EPISODE:  14
Score:  7
State count : 13
EPISODE:  15
Score:  9
State count : 11
EPISODE:  16
Score:  9
State count : 11
EPISODE:  17
Score:  5
State count : 15
EPISODE:  18
EPISODE:  19
Score:  11
State count : 9
EPISODE:  20
Score:  14
State count : 6
EPISODE:  21
Score:  3
State count : 17
EPISODE:  22
Score:  3
State count : 17
EPISODE:  23
Score:  8
State count : 12
EPISODE:  24
Score:  10
State count : 10
EPISODE:  25
Score:  7
State count